In [10]:
import numpy as np
import pandas as pd

import helper_functions 
from helper_functions import *
from imp import reload
import pandas as pd
import data_formatting
from data_formatting import *
reload(helper_functions)
reload(data_formatting)

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

def round_with_padding(value, round_digits):
    return format(round(value,round_digits), "."+str(round_digits)+"f")

csv_path='/home/ubuntu/enigma/motor_predictions/Behaviour_Information_ALL_April7_2022_sorted_CST_12_ll_slnm.csv'
nemo_settings='1mm,sdstream'
nemo_path='/home/ubuntu/enigma/lesionmasks/'
results_path='/home/ubuntu/enigma/results/'
analysis_id='analysis_1'
workbench_vis=False
scenesdir='/home/ubuntu/enigma/motor_predictions/wb_files'
hcp_dir='/home/ubuntu/enigma/motor_predictions/wb_files/HCP_S1200_GroupAvg_v1'
wbpath='/home/ubuntu/enigma/motor_predictions/wb_files/workbench_ubuntu/bin_linux64'
chaco_type='chacovol'
atlas='fs86subj'
covariates=['AGE','SEX','DAYS_POST_STROKE']
boxplots=False
lesionload_types='M1,all,all_2h'
crossval_types='1'
verbose=0
nperms=100
save_models=True
ensembles='none'
subid_colname='BIDS_ID'
ensemble_atlas='fs86subj'
site_colname='SITE'
chronicity_colname='CHRONICITY'
yvar_colname='NORMED_MOTOR'
subset='acute'
y_var='normed_motor_scores'
model_specified='ridge'
override_rerunmodels=False
remove_demog=None
ll='M1'
nemo_settings= ['1mm','sdstream']


original_dataset = load_csv(csv_path)
return_motor= True

[X, Y, C, lesion_load, site,site_idx, lesioned_hem,motor_vars] = create_data_set(csv_path, site_colname, nemo_path,yvar_colname, subid_colname,chronicity_colname,atlas, covariates, verbose, y_var,chaco_type, subset, remove_demog , nemo_settings, ll,return_motor)




Loading .csv...
/home/ubuntu/enigma/motor_predictions/Behaviour_Information_ALL_April7_2022_sorted_CST_12_ll_slnm.csv

Size of dataset before removing subjects without outcome scores: 1068 subjects
Size of dataset after removing subjects without outcome scores: 1020 subjects


Size of dataset before removing subjects without non-lesion covariates ['AGE', 'SEX', 'DAYS_POST_STROKE']: 1020 subjects
Size of dataset after removing subjects without non-lesion covariates: 792 subjects

Size of dataset before subsetting for chronic/acute: 791 subjects

Selecting acute subjects only. 

Size of dataset after subsetting for chronic/acute: 329 subjects
fs86subj
chacovol
_ses1_space-MNI152_desc-T1-lesion_mask_MNI
Multiple different image file names.
_ses1_space-MNI152_desc-T1-lesion_mask_MNI
Multiple different image file names.

The following subjects are in the .csv file but do not have corresponding ChaCo data: ['sub-r039s001', 'sub-r039s003']

(327, 86)
Index(['Unnamed: 0', 'BIDS_ID', 'SES', '

In [11]:
np.count_nonzero((lesioned_hem!=1) & (lesioned_hem!=2))

43

In [128]:
siteID = np.unique(site)
fmStrings=[]
aratStrings=[]
gripStrings=[]
nihssStrings=[]
wmftStrings=[]
barthelStrings=[]
motoricStrings=[]
nTotal=[]
fmsum=0
# get mesaures for each site
for sites in siteID:
    idx = site==sites
    motor_site = motor_vars[idx]
    nTotal.append(np.count_nonzero(idx))
    if pd.Series.equals(motor_site['FUGL_MEYER_UE_NORM'],motor_site['NORMED_MOTOR']):
        fmString = "{}".format(np.count_nonzero(idx))
        fmsum = fmsum+ np.count_nonzero(idx)
        fmStrings.append(fmString)
        aratString = "0"
        aratStrings.append(aratString)
        barthelString ="0"
        barthelStrings.append(barthelString)
        gripString = "0"
        gripStrings.append(gripString)
        motoricString = "0"
        motoricStrings.append(motoricString)
        nihssString ="0"
        nihssStrings.append(nihssString)
        wmftString ="0"
        wmftStrings.append(wmftString)
        continue
    else:
        aratString = "{}".format(np.count_nonzero(~np.isnan(motor_site['ARAT_NORM'])))
        aratStrings.append(aratString)
        barthelString ="{}".format(np.count_nonzero(~np.isnan(motor_site['BARTHEL'])))
        barthelStrings.append(barthelString)
        gripString ="{}".format(np.count_nonzero(~np.isnan(motor_site['GRIP_NORM_A_TO_NA'])))
        gripStrings.append(gripString)
        fmString = "{}".format(np.count_nonzero((~np.isnan(motor_site['FUGL_MEYER_UE_NORM']))))
        fmStrings.append(fmString)
        motoricString = "{}".format(np.count_nonzero(~np.isnan(motor_site['MOTRICITY_ARM_A'])))
        motoricStrings.append(motoricString)
        nihssString ="{}".format(np.count_nonzero(~np.isnan(motor_site['NIHSS_NORM'])))
        nihssStrings.append(nihssString)
        wmftString ="{}".format(np.count_nonzero(~np.isnan(motor_site['WMFT_FAS_NORM'])))
        wmftStrings.append(wmftString)

       

In [115]:

datatable = pd.DataFrame({'Total N.':nTotal,'Fugl Meyer UE':fmStrings,  'Barthel':barthelStrings, 'NIHSS':nihssStrings, 'ARAT':aratStrings})
#print(datatable.to_latex(index=False))

datatable.index = site_idx

In [116]:

s = datatable.style
print(s.to_latex(
    position="h", position_float="centering",
    hrules=True, label="table:5", caption="Type of test assessed at each site",
    multirow_align="t",multicol_align="c"
)  )

\begin{table}[h]
\centering
\caption{Type of test assessed at each site}
\label{table:5}
\begin{tabular}{lrllll}
\toprule
 & Total N. & Fugl Meyer UE & Barthel & NIHSS & ARAT \\
\midrule
r005 & 1 & 1 & 0 & 0 & 0 \\
r009 & 50 & 0 & 0 & 49 & 0 \\
r025 & 9 & 0 & 0 & 0 & 0 \\
r028 & 1 & 1 & 0 & 0 & 0 \\
r031 & 36 & 36 & 0 & 0 & 0 \\
r038 & 72 & 0 & 72 & 0 & 0 \\
r040 & 57 & 0 & 57 & 0 & 0 \\
r047 & 2 & 2 & 0 & 0 & 0 \\
r049 & 21 & 0 & 0 & 21 & 0 \\
r050 & 14 & 0 & 0 & 14 & 0 \\
r053 & 52 & 0 & 0 & 0 & 0 \\
r054 & 12 & 0 & 0 & 12 & 12 \\
\bottomrule
\end{tabular}
\end{table}

